In [ ]:
#Imports
import os
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
import xarray as xr
import folium
import numpy as np
import pandas as pd
import datetime as dt

from eodag import EODataAccessGateway
from eodag import setup_logging

from rasterio.crs import CRS
from rioxarray.merge import merge_arrays

from eotools.shortcut import prepare, configure, deserialize

# Setup Verbose Values:
# 0: no logging and no progress bar
# 1: no logging but progress bars displayed
# 2: log at the INFO level
# 3: log at the DEBUG level (even more information)

setup_logging(verbose=0)


# EODAG - merging

EODAG (Earth Observation Data Access Gateway) is a command line tool and a Python package for searching and downloading remotely sensed images while offering a unified API for data access regardless of the data provider.

EODAG gives you an easy way to access products from more than 10 providers, with more than 50 different product types (Sentinel 1, Sentinel 2, Sentinel 3, Landsat, etc.) that can be searched and downloaded.

## Data Acquisition
The Lines of Code in this chapter, which essentialy load the data, are not further explained here, since the Notebooks 01 and 02 do that.

In [ ]:
# Get Credentials from .env file and make dirs from paths.yml
# These functions are just shortcuts from a python script and just help to keep the notebooks short and simple
secrets, ws_paths = prepare(log=False)
dag = configure(secrets=secrets, paths=ws_paths)
deserialized_search_results = deserialize(filepath="search_results.geojson", ws_path=ws_paths, dag=dag, log=True)

# Download multiple Products
products = deserialized_search_results
paths = dag.download_all(products)

In [ ]:
# Set Boundingbox for Area inside the Tile.
latmin, latmax = 48.1, 48.35
lonmin, lonmax = 16.1, 16.6
extent = {'lonmin': lonmin, 'latmin': latmin, 'lonmax': lonmax, 'latmax': latmax}

# Folium Map
fmap = folium.Map(location=(np.array([latmin, latmax]).mean(), np.array([lonmin, lonmax]).mean()), zoom_start=9)
folium.Rectangle(bounds=[[latmin, lonmin],[latmax, lonmax]], color="red").add_to(fmap)
folium.GeoJson(
    data=products[:],  # SearchResult has a __geo_interface__ interface used by folium to get its GeoJSON representation, single results dont work (this [2:3] instead of [2])
    tooltip=folium.GeoJsonTooltip(fields=["title"])
).add_to(fmap)
fmap

In [ ]:
# Setting common Parameters for all further image processing
common_params = dict(
    crs=CRS.from_epsg(4326),               # the downloaded images are in 4326, don't reproject them
    resolution=0.0006,                     # but lower their resolution (0.0006 should be 60m in 100km)
    extent=(lonmin,latmin,lonmax,latmax)   # and zoom over/crop the area of interest
)

# Define basic Functions for future operations
def normalized_difference(a, b):
    return (a - b*1.)/(a + b)

## Post Process - Merging

In [ ]:
# Get a list of all available Bands (assets)
def get_assets(root:str, res=60):
    jp2_files = [file for dirs in os.walk(root, topdown=True)
                     for file in dirs[2] if file.endswith(f"_{res}m.jp2")]
    assets = [file.split('_')[2] for file in jp2_files if file.startswith('T')]
    return assets
    

bands_to_load = get_assets(paths[0], res=10)[1:-1]
print(bands_to_load)


In [ ]:
# Functions for loading data into datasets.

def load_single_product(product, bands:list):
    loaded_data = {}

    for band in bands:
        data = product.get_data(band=band, **common_params)
        data = data.squeeze()

        time_str = product.properties['startTimeFromAscendingNode']
        date = dt.datetime.strptime(time_str,'%Y-%m-%dT%H:%M:%S.%f%z')

        data = data.expand_dims(dim={'time':[date.date()]})
        data.name = band
        loaded_data[band] = data
    ds = xr.Dataset(loaded_data)
    return ds

def load_multiple_timestamps(products, bands:list):
    single_ds = []
    for product in products:
        single_product = load_single_product(product=product, bands=bands)
        single_ds.append(single_product)

    ds = xr.merge(single_ds)
    return ds

In [ ]:
# Loading multiple Bands into a dataset with multiple Timestamps of single Tile

tile_33UWP = sorted([p for p in products if 'T33UWP' == p.properties['title'].split('_')[5]], key=lambda p: p.properties["title"].split("_")[2])

ds_33UWP = load_multiple_timestamps(products=tile_33UWP, bands=bands_to_load)
print(f'This dataset from Tile T33UWP has {len(ds_33UWP.time)} timestamps.')

In [ ]:
# Loading multiple Bands into a dataset with multiple Timestamps of single Tile

tile_33UXP = sorted([p for p in products if 'T33UXP' == p.properties['title'].split('_')[5]], key=lambda p: p.properties["title"].split("_")[2])

ds_33UXP = load_multiple_timestamps(products=tile_33UXP, bands=bands_to_load)
print(f'This dataset from Tile T33UXP has {len(ds_33UXP.time)} timestamps.')

In [ ]:
# The Merging of the datasets (tile_33UXP and tile_33UWP) (each with multiple timestamps on two different tiles) has not yet been implemented.

#### Merging of single Arrays

In this example two geographicaly overlapping images are merged, and the NDVI is calculated. Note that only Dataarrays are merged but not Datasets.

In [ ]:
# Sort Products 
products = sorted([p for p in deserialized_search_results], key=lambda p: p.properties["title"].split("_")[2])
products[0:2]

In [ ]:
# Load Data of Bands 4 and 8. The Images are from the same time, but different tiles.
RED_1 = products[0].get_data(band="B04", **common_params)
RED_2 = products[1].get_data(band="B04", **common_params)

NIR_1 = products[0].get_data(band="B08", **common_params)
NIR_2 = products[1].get_data(band="B08", **common_params)

In [ ]:
# Using merge_arrays returns the geospatially merged data.
merged_red = merge_arrays([RED_1, RED_2], method='max') # methods = ['first', 'last', 'min', 'max', 'sum', 'count']
merged_nir = merge_arrays([NIR_1, NIR_2], method='max')

merged_ndvi = normalized_difference(merged_nir, merged_red)

In [ ]:
merged_ndvi.plot(cmap='RdYlGn', center=False, size=6, aspect='auto')